# SpaCy

SpaCy est une alternative à NLTK. 

In [3]:
import numpy as np
import pandas as pd
# pour ajouter des barres de progression sur les boucles
from tqdm import tqdm

import string
import matplotlib.pyplot as plt

from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE

import concurrent.futures
import time
# des représentation graphiques
import pyLDAvis.sklearn

from pylab import bone, pcolor, colorbar, plot, show, rcParams, savefig
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
import os
#print(os.listdir("../input"))

# Dataviz avec plotly
from plotly import tools
import plotly.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.figure_factory as ff

# import liés à spaCy
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

In [8]:
# installation de en_core_web_lg
# English multi-task CNN trained on OntoNotes, with GloVe vectors trained on Common Crawl. 
# Assigns word vectors, context-specific token vectors, POS tags, dependency parse and named entities.
#!python -m spacy download en_core_web_lg

In [4]:
# chargement des données
wines = pd.read_csv('./data/winemag-data_first150k.csv.zip')
wines.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude


In [15]:
wines.shape

(150930, 11)

In [5]:
# création d'un objet SpaCy
nlp = spacy.load('en_core_web_lg')

### Reconnaissance d'entités nommées

La reconnaissance d'entités nommées est une tâche d'extraction d'informations dans laquelle des entités nommées dans des phrases non structurées 
sont localisées et classées dans des catégories prédéfinies telles que les noms de personnes, les organisations, les lieux, les codes médicaux, 
les expressions de temps, les quantités, les valeurs monétaires, les pourcentages, etc.

In [22]:
doc = nlp(wines["description"][3])
spacy.displacy.render(doc, style='ent',jupyter=True)

In [23]:
punctuations = string.punctuation
stopwords = list(STOP_WORDS)


### Lemmatisation
C'est le processus de regroupement des formes infléchies d'un mot afin qu'elles puissent être analysées comme un seul élément. 

Des mots tels que "run" et "running" sont convertis en "run" pour éviter que des mots ayant des significations similaires dans nos données.

In [8]:
review = str(" ".join([i.lemma_ for i in doc]))

In [13]:
doc = nlp(review)
spacy.displacy.render(doc, style='ent',jupyter=True)


### Balisage (Parts of Speech tagging)
C'est le processus qui consiste à marquer un mot dans un texte (corpus) comme correspondant à une partie particulière du discours, en fonction à la fois de sa définition et de son contexte, c’est-à-dire de sa relation avec les mots adjacents et liés dans une phrase, une phrase. , ou paragraphe. 

Une forme simplifiée de ceci est couramment enseignée aux enfants d’âge scolaire, dans l’identification des mots en tant que noms, verbes, adjectifs, adverbes, etc.

In [14]:
# POS tagging
for i in nlp(review):
    print(i,"=>",i.pos_)

this => DET
spend => NOUN
20 => NUM
month => NOUN
in => ADP
30 => NUM
% => NOUN
new => ADJ
french => ADJ
oak => NOUN
, => PUNCT
and => CCONJ
incorporate => VERB
fruit => NOUN
from => ADP
ponzi => NOUN
's => PART
aurora => NOUN
, => PUNCT
abetina => ADJ
and => CCONJ
madrona => NOUN
vineyard => NOUN
, => PUNCT
among => ADP
other => ADJ
. => PUNCT
aromatic => ADJ
, => PUNCT
dense => ADJ
and => CCONJ
toasty => ADJ
, => PUNCT
-PRON- => PUNCT
deftly => ADV
blend => VERB
aroma => NOUN
and => CCONJ
flavor => NOUN
of => ADP
toast => NOUN
, => PUNCT
cigar => NOUN
box => NOUN
, => PUNCT
blackberry => NOUN
, => PUNCT
black => ADJ
cherry => NOUN
, => PUNCT
coffee => NOUN
and => CCONJ
graphite => NOUN
. => PUNCT
tannin => NOUN
be => VERB
polish => NOUN
to => ADP
a => DET
fine => ADJ
sheen => NOUN
, => PUNCT
and => CCONJ
frame => VERB
a => DET
finish => NOUN
load => NOUN
with => ADP
dark => ADJ
chocolate => NOUN
and => CCONJ
espresso => NOUN
. => PUNCT
drink => VERB
now => ADV
through => ADP
2032 => 

In [26]:
# Parser for reviews
parser = English()
def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]
    mytokens = " ".join([i for i in mytokens])
    return mytokens

In [27]:
tqdm.pandas()
wines["processed_description"] = wines["description"].progress_apply(spacy_tokenizer)

100%|███████████████████████████████████████████████| 150930/150930 [03:00<00:00, 836.20it/s]


In [28]:
wines["processed_description"]

0         tremendous 100 varietal wine hail oakville age...
1         ripe aroma fig blackberry cassis soften sweete...
2         mac watson honor memory wine mother tremendous...
3         spend 20 month 30 new french oak incorporate f...
4         wine la bégude high point vineyard 1200 foot s...
5         deep dense pure open bell toro winner aromas d...
6         slightly gritty black fruit aroma include swee...
7         lush cedary black fruit aroma luxe offer note ...
8         vineyard bottle delancellotti find strike mine...
9         producer source block vineyard wine — high ele...
10        elegance complexity structure come drop dead g...
11        18-year old vine supple good balance effort bl...
12        standout terrific lineup 2015 release patricia...
13        wine peak condition tannin secondary flavor do...
14        sophisticate mix mineral acid tart fruit seduc...
15        2006 succulent luscious chardonnay minerality ...
16        blockbuster powerhouse wine su

# Qu'est-ce que le  topic-modelling ?

Dans l’apprentissage automatique et le traitement du langage naturel, un topic modeling est un type de modèle statistique permettant de découvrir les "sujets" abstraits qui apparaissent dans une collection de documents. 

La modélisation de sujets est un outil d'exploration de texte fréquemment utilisé pour la découverte de structures sémantiques cachées dans un corps de texte. 

Intuitivement, étant donné qu’un document porte sur un sujet particulier, on peut s’attendre à ce que des mots particuliers apparaissent plus ou moins fréquemment dans le document: "chien" et "os" apparaissent plus souvent dans les documents relatifs aux chiens, "chat" et "miaou" apparaîtra dans les documents sur les chats, et "le" et "est" apparaîtra également dans les deux. 

Un document concerne généralement plusieurs sujets dans des proportions différentes; ainsi, dans un document qui parle de 10% de chats et de 90% de chiens, il y aurait probablement environ 9 fois plus de mots de chien que de mots de chat.

Les "sujets" produits par les techniques de modélisation de sujets sont des groupes de mots similaires. Un modèle de sujet capture cette intuition dans un cadre mathématique, ce qui permet d'examiner un ensemble de documents et de découvrir, sur la base de la statistique des mots de chaque mot, quels sont les sujets et quel est le solde des sujets de chaque document. Il implique diverses techniques de réduction de dimensionnalité (généralement non linéaire) et d’apprentissage non supervisé, telles que LDA, SVD, autoencodeurs, etc.

In [29]:
# Creating a vectorizer
vectorizer = CountVectorizer(min_df=5, max_df=0.9, stop_words='english', lowercase=True, token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(wines["processed_description"])

In [30]:
NUM_TOPICS = 10

In [31]:
# Latent Dirichlet Allocation Model
lda = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=10, learning_method='online',verbose=True)
data_lda = lda.fit_transform(data_vectorized)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


In [32]:
# Non-Negative Matrix Factorization Model
nmf = NMF(n_components=NUM_TOPICS)
data_nmf = nmf.fit_transform(data_vectorized) 

In [24]:
# Latent Semantic Indexing Model using Truncated SVD
lsi = TruncatedSVD(n_components=NUM_TOPICS)
data_lsi = lsi.fit_transform(data_vectorized)

In [25]:
# Functions for printing keywords for each topic
def selected_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]]) 

In [26]:
# Keywords for topics clustered by Latent Dirichlet Allocation
print("LDA Model:")
selected_topics(lda, vectorizer)

LDA Model:
Topic 0:
[('wine', 16153.212881090501), ('oak', 14107.632735455003), ('good', 8849.326773604449), ('little', 8682.630195700372), ('flavor', 7578.359871471053), ('vintage', 5396.188980383416), ('new', 5300.363142913051), ('price', 4984.582847892463), ('sweet', 4674.156452619467), ('tannic', 4492.910872999984)]
Topic 1:
[('cherry', 20242.05538213588), ('flavor', 17730.56148110044), ('raspberry', 9565.195626628944), ('pinot', 8256.737931549826), ('spin-dry', 8186.4337230696565), ('wine', 8030.264059960273), ('colon', 7606.319526349391), ('drink', 6986.996476683885), ('soft', 6043.689168734124), ('pretty', 5945.335315425528)]
Topic 2:
[('wine', 25876.279987653495), ('tannin', 22488.24437194703), ('fruit', 21640.662202483803), ('year', 15066.136224471635), ('black', 13190.886809359861), ('cabernet', 11649.761390724447), ('ripe', 10694.641684970056), ('flavor', 10228.838006757207), ('age', 9763.430970711786), ('structure', 9633.148883128772)]
Topic 3:
[('flavor', 10849.18160307530

In [27]:
# Keywords for topics clustered by Latent Semantic Indexing
print("NMF Model:")
selected_topics(nmf, vectorizer)

NMF Model:
Topic 0:
[('flavor', 20.78933359750987), ('spin-dry', 2.242680712249387), ('sweet', 1.7018079018698056), ('oak', 1.6598327768105554), ('vanilla', 1.0834122485189581), ('little', 1.046871585337875), ('blackberry', 0.9715405365856767), ('finish', 0.9517528294026383), ('like', 0.8681329836004484), ('bite', 0.866439992041679)]
Topic 1:
[('wine', 15.729456358775007), ('age', 0.9881832522110268), ('year', 0.7143573167488565), ('spice', 0.5790605975996849), ('character', 0.5209066706286741), ('structure', 0.5184371330022532), ('wood', 0.48401939702969204), ('texture', 0.4723475940609506), ('great', 0.45668470179616744), ('like', 0.34258719107905533)]
Topic 2:
[('fruit', 16.50550105664452), ('red', 1.2719647928664506), ('wood', 0.6423501990494488), ('tropical', 0.6059791538223456), ('barrel', 0.5740832289641595), ('spice', 0.4921797041106512), ('stone', 0.47962087195731756), ('oak', 0.43709709886926673), ('hint', 0.38923965717284664), ('age', 0.3870074366715245)]
Topic 3:
[('tannin'

In [28]:
# Keywords for topics clustered by Non-Negative Matrix Factorization
print("LSI Model:")
selected_topics(lsi, vectorizer)

LSI Model:
Topic 0:
[('wine', 0.46545443701761935), ('flavor', 0.3786947502413103), ('fruit', 0.3487793778736476), ('finish', 0.18255713029753912), ('cherry', 0.17715056621518996), ('tannin', 0.15778296017605356), ('aroma', 0.15052434417332453), ('good', 0.1442278426005325), ('acidity', 0.1429670318217277), ('black', 0.1303079941681807)]
Topic 1:
[('wine', 0.6964312130724919), ('fruit', 0.1693109722555911), ('age', 0.09127965366894722), ('acidity', 0.06367300030183036), ('structure', 0.048584327594470574), ('year', 0.04829022561719037), ('wood', 0.04650128237418289), ('character', 0.040823043800071845), ('great', 0.03063915081850436), ('rich', 0.02969883906677463)]
Topic 2:
[('fruit', 0.7155225235079223), ('black', 0.16290458583205175), ('tannin', 0.11285731751109022), ('palate', 0.10293577397071027), ('aroma', 0.09501479918531289), ('note', 0.0836988407662544), ('berry', 0.07060461054014815), ('dark', 0.06665826873173686), ('finish', 0.0647699796684008), ('red', 0.06404920532673951)]


In [29]:
# Transforming an individual sentence
text = spacy_tokenizer("Aromas include tropical fruit, broom, brimstone and dried herb. The palate isn't overly expressive, offering unripened apple, citrus and dried sage alongside brisk acidity.")
x = lda.transform(vectorizer.transform([text]))[0]
print(x)

[0.00500012 0.00500033 0.00500038 0.06546978 0.00500121 0.08337955
 0.19314925 0.00500002 0.45912628 0.17387309]


# Visualisation avec pyLDAvis

In [30]:
pyLDAvis.enable_notebook()
dash = pyLDAvis.sklearn.prepare(lda, data_vectorized, vectorizer, mds='tsne')
dash

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
2      -87.654572  -85.905556       1        1  15.757019
6       19.383211 -111.420372       2        1  13.321650
5      113.906326  -26.348547       3        1  12.517125
9      -12.132884   -6.143361       4        1  11.564259
1      116.255463 -166.908035       5        1  10.593863
0     -190.506821  -37.415215       6        1  10.306650
8     -149.103928 -181.399399       7        1   8.899201
3     -106.279121   62.855598       8        1   7.256164
4       39.861206   92.573227       9        1   6.351241
7      -23.617956 -220.064972      10        1   3.432828, topic_info=      Category          Freq        Term         Total  loglift  logprob
term                                                                    
1837   Default  40706.000000      cherry  40706.000000  30.0000  30.0000
514    Default  34128.000000       aroma  34128.000000  29.0000  29.0000
11163  Default  95319.000000        wine  95319.000000  28.0000  28.0000
6826   Default  20648.000000         oak  20648.000000  27.0000  27.0000
9939   Default  33000.000000      tannin  33000.000000  26.0000  26.0000
3893   Default  85184.000000      flavor  85184.000000  25.0000  25.0000
3830   Default  41903.000000      finish  41903.000000  24.0000  24.0000
923    Default  21355.000000       berry  21355.000000  23.0000  23.0000
11260  Default  16154.000000        year  16154.000000  22.0000  22.0000
7109   Default  27541.000000      palate  27541.000000  21.0000  21.0000
986    Default  25027.000000       black  25027.000000  20.0000  20.0000
446    Default  13567.000000       apple  13567.000000  19.0000  19.0000
7550   Default  14991.000000        plum  14991.000000  18.0000  18.0000
4133   Default  73080.000000       fruit  73080.000000  17.0000  17.0000
89     Default  30617.000000     acidity  30617.000000  16.0000  16.0000
1444   Default  11597.000000    cabernet  11597.000000  15.0000  15.0000
6777   Default  23970.000000        note  23970.000000  14.0000  14.0000
8029   Default  12217.000000   raspberry  12217.000000  13.0000  13.0000
1952   Default  12511.000000      citrus  12511.000000  12.0000  12.0000
5736   Default  10946.000000      little  10946.000000  11.0000  11.0000
4412   Default  32576.000000        good  32576.000000  10.0000  10.0000
7238   Default  10807.000000       peach  10807.000000   9.0000   9.0000
7470   Default   9535.000000       pinot   9535.000000   8.0000   8.0000
976    Default  10870.000000        bite  10870.000000   7.0000   7.0000
9354   Default  22866.000000    spin-dry  22866.000000   6.0000   6.0000
4985   Default   6859.000000    imported   6859.000000   5.0000   5.0000
2122   Default   7496.000000       colon   7496.000000   4.0000   4.0000
11117  Default  12285.000000       white  12285.000000   3.0000   3.0000
4101   Default  17868.000000       fresh  17868.000000   2.0000   2.0000
9341   Default  27807.000000       spice  27807.000000   1.0000   1.0000
...        ...           ...         ...           ...      ...      ...
11216  Topic10    892.292969       woody    893.188053   3.3708  -4.8783
5716   Topic10    882.890797     liqueur    883.785890   3.3708  -4.8889
1245   Topic10    860.230133      briary    861.125224   3.3707  -4.9149
3107   Topic10    852.160100         dre    853.055089   3.3707  -4.9243
6502   Topic10    847.889263       mourv    848.784253   3.3707  -4.9293
5797   Topic10    837.337334        lose    838.232438   3.3707  -4.9418
7197   Topic10    830.374009        past    831.269102   3.3707  -4.9502
8680   Topic10    790.091758    saturate    790.986862   3.3707  -4.9999
1982   Topic10    775.506789       class    776.401927   3.3706  -5.0186
7667   Topic10    738.498603      powder    739.393683   3.3706  -5.0675
8952   Topic10    738.316245       sheer    739.211339   3.3706  -5.0677
8259   Topic10    702.988859    restrain    703.8

## Comment interpréter ce graphique?
1. Sujets à gauche tandis que leurs mots-clés respectifs sont à droite.
2. Les sujets plus grands sont plus fréquents et plus proches les uns des autres, plus la similitude
3. La sélection des mots-clés dépend de leur fréquence et de leur caractère discriminant.

** Survolez les rubriques à gauche pour obtenir des informations sur les mots clés à droite. **